# Exporting a model for deployment

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
# import window_generator as WindowGenerator
from window_generator import WindowGenerator

# Loading the dataset

In [3]:
df = pd.read_csv('extracted_data/csv/hour_merged.csv',
                 parse_dates={'data': ['date', 'time']},
                 infer_datetime_format=True,
                 dayfirst=False,
                 index_col='data'
                )


df.head()

,price,demand,emissions
data,,,
2017-01-01 00:00:00,58.82,24682,14467.2840
2017-01-01 01:00:00,58.23,24046,14173.8295
2017-01-01 02:00:00,51.95,22665,13198.0690
2017-01-01 03:00:00,47.27,21200,12510.7595
2017-01-01 04:00:00,45.49,20056,12203.7315


In [4]:
# Select data from 2021
df = df[int((len(df)*0.8)):]

# Differencing

In [5]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [6]:
diff_price = difference(df['price'])
diff_emissions = difference(df['emissions'])
diff_demand = difference(df['demand'])

In [7]:
s1 = pd.Series([0.0])
diff_price = s1.append(diff_price, ignore_index=True)
diff_emissions = s1.append(diff_emissions, ignore_index=True)
diff_demand = s1.append(diff_demand, ignore_index=True)

In [8]:
df['price'] = diff_price.values
df['demand'] = diff_demand.values
df['emissions'] = diff_emissions.values

# Split into training and test

In [9]:
# 80% for training and 20% for test
n = len(df)
train_df = df[0:int(n*0.8)]
test_df = df[int(n*0.8):]

# Normalization/Standarization

In [10]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

# Training a model

In [11]:
MAX_EPOCHS = 20
patience = 4
num_features=df.shape[1]
OUT_STEPS = 24

In [12]:
multi_lstm_model = tf.keras.Sequential([
        # Shape [batch, time, features] => [batch, lstm_units]
        tf.keras.layers.LSTM(128, return_sequences=False),
        # Shape [batch, out_steps*features]
        tf.keras.layers.Dense(OUT_STEPS*num_features, kernel_initializer=tf.initializers.zeros()),
        tf.keras.layers.Dropout(.1),
        # Shape [batch, out_steps, features]
        tf.keras.layers.Reshape([OUT_STEPS, num_features])
    ])
    
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    
multi_lstm_model.compile(loss='MeanSquaredError',
                 optimizer='rmsprop',
                 metrics=[tf.metrics.RootMeanSquaredError()])
    
# Creating a Window
window = WindowGenerator(input_width=24, label_width=OUT_STEPS, shift=OUT_STEPS, train_df=train_df, test_df=test_df)
    
# History
history = multi_lstm_model.fit(window.train, epochs=MAX_EPOCHS, validation_data = window.test, 
                        callbacks=[early_stopping]
                       )
    
print(multi_lstm_model.evaluate(window.test, verbose=0))
loss, rmse = multi_lstm_model.evaluate(window.test, verbose=0)

Epoch 1/20
218/218 [==============================] - 5s 16ms/step - loss: 0.6743 - root_mean_squared_error: 0.8211 - val_loss: 0.8489 - val_root_mean_squared_error: 0.9213
Epoch 2/20
218/218 [==============================] - 3s 16ms/step - loss: 0.5720 - root_mean_squared_error: 0.7563 - val_loss: 0.7714 - val_root_mean_squared_error: 0.8783
Epoch 3/20
218/218 [==============================] - 4s 16ms/step - loss: 0.5190 - root_mean_squared_error: 0.7204 - val_loss: 0.7049 - val_root_mean_squared_error: 0.8396
Epoch 4/20
218/218 [==============================] - 4s 17ms/step - loss: 0.4827 - root_mean_squared_error: 0.6948 - val_loss: 0.6871 - val_root_mean_squared_error: 0.8289
Epoch 5/20
218/218 [==============================] - 4s 17ms/step - loss: 0.4575 - root_mean_squared_error: 0.6764 - val_loss: 0.6868 - val_root_mean_squared_error: 0.8288
Epoch 6/20
218/218 [==============================] - 3s 15ms/step - loss: 0.4319 - root_mean_squared_error: 0.6572 - val_loss: 0.6908 

In [13]:
multi_lstm_model.save('saved_model/')

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [14]:
loaded_model = keras.models.load_model('saved_model/')

In [15]:
print(loaded_model.evaluate(window.test, verbose=0))

[0.6139790415763855, 0.7835680842399597]
